In [1]:
from selenium.webdriver.common.by import By
from selenium import webdriver
import time
import pandas as pd

##### You should specify the path to the chromedriver that is located in your computer and assign it to the path variable below.

In [2]:
def enter_website():
    global driver
    url = 'https://en.bidfax.info/'
    path = '/Users/areg/.wdm/drivers/chromedriver/mac64/108.0.5359/chromedriver'
    # You should specify the path of the chromedriver in the brackets 
    # that are in front of path variable above
    driver = webdriver.Chrome(path)
    driver.get(url)

#### After calling select_make() function it will ask you to input a make of a car and then it will select it. If that make will not exit it will tell you to put the existing one.

In [3]:
def select_make():
    dropdown = driver.find_elements(By.XPATH, '//span[@class = "drop-down"]' )
    make = str(input('Please write the make you want to select:  '))
    def search_for_make(make):
        try:
            dropdown[0].click()
        except:
            pass
        try:
            make_ = driver.find_element(By.LINK_TEXT, make)
            make_.click()
            print(f"{make} make is selected successfully!")
        except:
            print(f'{make} make does not exist in this list. Please try again.')
            return select_make()
    search_for_make(make)
    time.sleep(1)

#### After calling select_model() fucntion it will ask you to input a model of a car and then it will select it. If that make will not exit it will tell you to put the existing one.

In [4]:
def select_model():
    dropdown = driver.find_elements(By.XPATH, '//span[@class = "drop-down"]' )
    model = str(input('Please write the model you want to select:  '))
    def search_for_model(model):
        try:
            dropdown[1].click()
        except:
            pass
        try:
            model_ = driver.find_element(By.LINK_TEXT, model)
            model_.click()
            print(f"{model} model is selected successfully!")
        except:
            print(f'{model} model does not exist in this list. Please try again.')
            return select_model()
    search_for_model(model)  
    time.sleep(1)

#### After calling select_year_start() function below it will ask you to input the starting year of a car you want to search for and then it will select it. If that make will not exit it will tell you to put the existing one. Call the select_year_start function after calling select_make() and select_model() functions

In [5]:
def select_year_start():
    def scrape_years():
        global years
        years_dropdown = driver.find_element(By.XPATH, '//div[@class = "filter_right"]')
        years = years_dropdown.find_elements(By.XPATH, ('./select'))
    scrape_years()
    start = str(input("Type in the starting year of the car:  "))
    def year_start(start):   
        year_start = years[0]
        try:
            year_start = year_start.find_elements(By.XPATH, ("./option"))
            for year in year_start:
                if year.text == start:
                    print(f"{start} year is selected successfully!")
                    return year.click()
            raise print(f"{start} year does not exist. Please try again.")
        except:
            return select_year_start()
    year_start(start)
    time.sleep(1)

#### After calling select_year_to() function it will ask you to input the ending year of a car you want to search for and then it will select it. If that make will not exit it will tell you to put the existing one.  Call the select_year_start function after calling select_make() and select_model() functions

In [6]:
def select_year_to():
    def scrape_years():
        global years
        years_dropdown = driver.find_element(By.XPATH, '//div[@class = "filter_right"]')
        years = years_dropdown.find_elements(By.XPATH, ('./select'))    
    scrape_years()
    to = str(input("Type in the ending year of the car:  "))
    def year_end(year_to):   
        year_to = years[1]
        try:
            year_to = year_to.find_elements(By.XPATH, ("./option"))
            for year in year_to:
                if year.text == to:
                    print(f"{to} year is selected successfully!")
                    return year.click()
            raise print(f"{to} year does not exist. Please try again.")
        except:
            return select_year_to()
    year_end(to)
    time.sleep(1)
    

In [7]:
def scrape_page():
    global data
        
    lot_number = []; date_of_sale = []; year = []; vin = []
    condition = []; engine = []; mileage = []; documents = []
    location = []; primary_damage = []; secondary_damage = []
    estimated_retail_value = [];estimated_repair_cost = []
    transmission = []; body_color = [];drive = []; fuel = []
    keys = []; notes = []; avg_price = []; vehicles = []
    price = []; market = []; seller = []
    
    def scrape_data():
        global info, info_p, auction, columns, data
        info = driver.find_elements(By.XPATH, '//span[@class = "blackfont"]')
        info_p = driver.find_element(By.XPATH, '//span[@class = "prices"]')
        auction = driver.find_element(By.XPATH, '//p[@class ="short-story"]/child::span')   


        data = [vehicles, avg_price, lot_number, date_of_sale, 
                year, vin, condition, engine, mileage,  
                documents, location, primary_damage, 
                secondary_damage, estimated_retail_value, 
                estimated_repair_cost, transmission, body_color,
                drive, fuel, keys, notes, seller, price, market] 
        columns = ['vehicles', 'avg_price', 'lot_number', 'date_of_sale', 
                   'year', 'vin', 'condition', 'engine', 'mileage', 
                   'documents', 'location', 'primary_damage', 'secondary_damage', 
                   'estimated_retail_value', 'estimated_repair_cost', 
                   'transmission', 'body_color','drive', 'fuel', 'keys', 
                   'notes','seller', 'price', 'market']   
    
    def adding_data_to_lists():
        scrape_data()
        price.append(info_p.text)
        market.append(auction.text)
        count_ = 0
        if auction.text == ' Copart ':
            data[21].append(info[9].text)
            for i in (info[:9]+info[10:22]):
                data[count_].append(i.text)
                count_ += 1  
        else:
            seller_ = driver.find_element(By.XPATH, '//p[@class = "iaaiseller"]')
            data[21].append(seller_.text)
            for i in info[:21]:
                data[count_].append(i.text)
                count_ += 1
    def go_in_car():
        driver.current_window_handle
        sold_cars_count = driver.find_elements(By.XPATH, '//img[@src = "https://bidfax.info/templates/ru/dleimages/sold2.png"]/../preceding-sibling::a')
        for i in range(len(sold_cars_count) - 1):
            sold_cars = driver.find_elements(By.XPATH, '//img[@src = "https://bidfax.info/templates/ru/dleimages/sold2.png"]/../preceding-sibling::a')
            sold_cars[i].click()
            time.sleep(3)
            adding_data_to_lists()
            driver.current_window_handle
            driver.back()
            time.sleep(3)
    go_in_car()


In [8]:
def create_df():
    scrape_page()
    df_data = {}
    for i, j in zip(columns, data):
        df_data[i]= j
    global df
    df = pd.DataFrame(df_data)
    time.sleep(10)


In [9]:
def add_next_pages(count):
    def next_page():
        next_page = driver.find_element(By.LINK_TEXT, '»')
        next_page.click()
        time.sleep(10)
    def add_to_df():
        scrape_page()
        df_data = {}
        global df
        for i, j in zip(columns, data):
            df_data[i] = j
        df1 = pd.DataFrame(df_data)
        df = df.append(df1)
        df = df.reset_index(drop = True)
    for i in range(int(count)):
        next_page()
        add_to_df()

In [10]:
enter_website()

/var/folders/7_/hrpf2wd52k91vfg1qd2m7ch80000gn/T/ipykernel_46267/877400425.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [11]:
select_make()

Volkswagen make is selected successfully!


In [12]:
select_model()

Gti model is selected successfully!


In [13]:
select_year_start()

2017 year is selected successfully!


In [14]:
select_year_to()

2020 year is selected successfully!


In [15]:
create_df()

In [16]:
add_next_pages(50)

/var/folders/7_/hrpf2wd52k91vfg1qd2m7ch80000gn/T/ipykernel_46267/1723988589.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df1)


KeyboardInterrupt: 

In [18]:
df

,vehicles,avg_price,lot_number,date_of_sale,year,vin,condition,engine,mileage,documents,...,estimated_repair_cost,transmission,body_color,drive,fuel,keys,notes,seller,price,market
0,224,$2288,37000783,08.02.2023,2013,WVWFV7AJ6DW005511,Run And Drive,2.0L 4,129887 miles (Actual),Ma - Certificate Of Title,...,0,Automatic,Gray,Front-Wheel Drive,Gas,Yes,Not specified,Not specified,2300,Copart
1,164,$4953,35950623,08.02.2023,2017,3VW447AU7HM064669,Run And Drive,2 L 4,78822 miles (Actual),Ca - Salvage Certificate,...,11872,Automatic,Black,Front-Wheel Drive,Gas,Yes,Not specified,Not specified,5800,Copart
2,36,$7442,72154222,07.02.2023,2020,3VW5T7AU0LM003406,Run And Drive,2.0L 4,38108 miles (Actual),Nj - Cert Of Title-Salvage,...,16851,Manual,White,Front-Wheel Drive,Gas,Yes,Not specified,Not specified,11400,Copart
3,164,$4953,65162572,06.02.2023,2017,3VW447AU0HM051049,Engine Start Program,2 L 4,41499 miles (Actual),Il - Salvage Certificate,...,21746,Automatic,White,Front-Wheel Drive,Gas,Yes,Not specified,Not specified,5200,Copart
4,164,$4953,72276762,02.02.2023,2017,3VW4T7AU2HM062928,Enhanced Vehicles,2 L 4,68244 miles (Actual),Ga - Cert Of Title-Salvage,...,21408,Automatic,Two Tone,Front-Wheel Drive,Gas,Yes,Not specified,Not specified,6400,Copart
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,51,$7098,64400602,22.11.2022,2019,3VW6T7AU7KM019153,Run And Drive,2.0L 4,14462 miles (Actual),Fl - Cert Of Title Slvg Rebuildable,...,7812,Automatic,White,Front-Wheel Drive,Gas,Yes,Not specified,Not specified,10100,Copart
116,164,$4953,61081252,22.11.2022,2017,3VW5T7AU2HM042912,Run And Drive,2 L 4,74211 miles (Actual),Ca - Salvage Certificate,...,15720,Manual,White,Front-Wheel Drive,Gas,Yes,Veh Loc @ San Martin Sublot,Not specified,2900,Copart
117,164,$4953,64654412,21.11.2022,2017,3VW5T7AU2HM022496,Run And Drive,2 L 4,96002 miles (Actual),On - Permit Salvage,...,26037,Manual,Gray,Front-Wheel Drive,Gas,Yes,Front/Minor Dents,Not specified,8800,Copart
118,164,$4953,64456022,21.11.2022,2017,3VW4T7AU4HM063479,Run And Drive,2 L 4,80796 miles (Actual),Fl - Certificate Of Destruction,...,19591,Automatic,Silver,Front-Wheel Drive,Gas,Yes,Not specified,Not specified,5200,Copart


In [20]:
df.to_csv('volkswagen_gti_2017-2020.csv', index = False)